<h2 style="color:lightgreen">Random Forests</h2>

### Generate n number of bootstraped datasets and store in forest

In [2]:
import pandas as pd
import numpy as np

In [3]:
def random_forest(dataset,num_trees):
	random_forest = []
	for i in range(num_trees):
		random_forest.append(dataset.sample(frac=1,replace=True))
	return random_forest

In [5]:
df = pd.read_csv("rajin.csv",header=None,names=['ID','P','R','Q','Class'])
r_forest = random_forest(df,5)